# 분류

## 서비스 이탈예측 데이터

In [13]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv")


display(x_train.head())
display(y_train.head())

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15799217,Zetticci,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
1,15748986,Bischof,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
2,15722004,Hsiung,543,France,Female,31,4,138317.94,1,0,0,61843.73
3,15780966,Pritchard,709,France,Female,32,2,0.00,2,0,0,109681.29
4,15636731,Ts'ai,714,Germany,Female,36,1,101609.01,2,1,1,447.73


,CustomerId,Exited
0,15799217,0
1,15748986,0
2,15722004,0
3,15780966,0
4,15636731,0


In [14]:
x_train.info(), x_train.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       6499 non-null   int64  
 1   Surname          6499 non-null   object 
 2   CreditScore      6499 non-null   int64  
 3   Geography        6499 non-null   object 
 4   Gender           6499 non-null   object 
 5   Age              6499 non-null   int64  
 6   Tenure           6499 non-null   int64  
 7   Balance          6499 non-null   float64
 8   NumOfProducts    6499 non-null   int64  
 9   HasCrCard        6499 non-null   int64  
 10  IsActiveMember   6499 non-null   int64  
 11  EstimatedSalary  6499 non-null   float64
dtypes: float64(2), int64(7), object(3)
memory usage: 609.4+ KB


(None, (6499, 12))

In [15]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

rf = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=0)
le = LabelEncoder()

In [16]:
## 필요없는 컬럼 삭제
x_train_drop = x_train.drop(columns=['Surname','CustomerId'])
x_test_drop = x_test.drop(columns=['Surname','CustomerId'])

In [18]:
## 레이블 인코딩
x_train_drop['Geography'] = le.fit_transform(x_train_drop['Geography'])
x_train_drop['Gender'] = le.fit_transform(x_train_drop['Gender'])

In [26]:
x_test_drop['Geography'] = le.fit_transform(x_test_drop['Geography'])
x_test_drop['Gender'] = le.fit_transform(x_test_drop['Gender'])

In [19]:
## train, test set split
from sklearn.model_selection import train_test_split

y = y_train.drop(columns='CustomerId')
X_train, X_val, y_train, y_val = train_test_split(x_train_drop, y, test_size=0.3, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(4549, 10) (1950, 10) (4549, 1) (1950, 1)


In [20]:
rf.fit(X_train, y_train)  # 학습

C:\Users\na\AppData\Local\Temp\ipykernel_15976\3956366250.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)  # 학습


RandomForestClassifier(max_depth=4, random_state=0)

In [24]:
## predict, 성능 평가
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, precision_score

predict_train = rf.predict(X_train)
predict_train_proba = rf.predict_proba(X_train)[:,1]
predict_val = rf.predict(X_val)
predict_val_proba = rf.predict_proba(X_val)[:,1]

In [25]:
# 문제에서 묻는 것에 따라 모델 성능 확인하기
# 정확도 (accuracy) , f1_score , recall , precision -> model.predict로 결과뽑기
# auc , 확률이라는 표현있으면 model.predict_proba로 결과뽑고 첫번째 행의 값을 가져오기 model.predict_proba()[:,1]
print('train accuracy :', accuracy_score(y_train, predict_train))
print('validation accuracy :', accuracy_score(y_val, predict_val))
print('\n')

print('train f1_score :', f1_score(y_train, predict_train))
print('validation accuracy :', f1_score(y_val, predict_val))
print('\n')

print('train recall_score :', recall_score(y_train, predict_train))
print('validation recall_score :', recall_score(y_val, predict_val))
print('\n')

print('train precision_score :', precision_score(y_train, predict_train))
print('validation precision_score :', precision_score(y_val, predict_val))
print('\n')

print('train auc :', roc_auc_score(y_train, predict_train_proba))
print('validation auc :', roc_auc_score(y_val, predict_val_proba))

train accuracy : 0.8542536821279402
validation accuracy : 0.8425641025641025


train f1_score : 0.47588932806324113
validation accuracy : 0.3747454175152749


train recall_score : 0.3188559322033898
validation recall_score : 0.24210526315789474


train precision_score : 0.9376947040498442
validation precision_score : 0.8288288288288288


train auc : 0.8544994005500834
validation auc : 0.8374907810928595


In [27]:
## test 데이터셋
predict_test = rf.predict(x_test_drop)
predict_test_proba = rf.predict_proba(x_test_drop)[:,1]

# accuracy, f1_score, recall, precision 
pd.DataFrame({'CustomerId': x_test.CustomerId, 'Exited': predict_test}).to_csv('2유형_1.csv', index=False)

# auc, 확률
pd.DataFrame({'CustomerId': x_test.CustomerId, 'Exited': predict_test_proba}).to_csv('2유형_1_proba.csv', index=False)

## 이직여부 판단 데이터

In [93]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv")


display(x_train.head())
display(y_train.head())

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,25298,city_138,0.836,Male,No relevent experience,Full time course,High School,NaN,5,100-500,Pvt Ltd,1,45
1,4241,city_160,0.920,Male,No relevent experience,Full time course,High School,NaN,5,NaN,NaN,1,17
2,24086,city_57,0.866,Male,No relevent experience,no_enrollment,Graduate,STEM,10,NaN,NaN,1,50
3,26773,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,>4,135
4,32325,city_143,0.740,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,17


,enrollee_id,target
0,25298,0.0
1,4241,1.0
2,24086,0.0
3,26773,0.0
4,32325,1.0


In [85]:
x_train.info(), x_train.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12452 entries, 0 to 12451
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             12452 non-null  int64  
 1   city                    12452 non-null  object 
 2   city_development_index  12452 non-null  float64
 3   gender                  9535 non-null   object 
 4   relevent_experience     12452 non-null  object 
 5   enrolled_university     12195 non-null  object 
 6   education_level         12137 non-null  object 
 7   major_discipline        10586 non-null  object 
 8   experience              12415 non-null  object 
 9   company_size            8600 non-null   object 
 10  company_type            8471 non-null   object 
 11  last_new_job            12179 non-null  object 
 12  training_hours          12452 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 1.2+ MB


(None, (12452, 13))

In [95]:
# print(x_train.nunique())
# 결측치 처리 X 더미화

# 범주형 변수인데 많은 unique값을 가진 컬럼 drop
drop_col = ['enrollee_id','city','company_type','experience']
x_train_drop = x_train.drop(columns = drop_col)
x_test_drop = x_test.drop(columns = drop_col)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

x_train_dummies = pd.get_dummies(x_train_drop)
x_test_dummies = pd.get_dummies(x_test_drop)
y = y_train['target'].astype('int')

# train, test 컬럼 순서 동일하게 만들기
x_test_dummies = x_test_dummies[x_train_dummies.columns]

## split
X_train, X_validation, Y_train, Y_validation = train_test_split(x_train_dummies, y, test_size=0.33, random_state=42)
rf = RandomForestClassifier(random_state =23)
rf.fit(X_train,Y_train)

RandomForestClassifier(random_state=23)

In [96]:
from sklearn.metrics import accuracy_score , f1_score, recall_score, roc_auc_score ,precision_score

#model_score
predict_train_label = rf.predict(X_train)
predict_train_proba = rf.predict_proba(X_train)[:,1]

predict_validation_label = rf.predict(X_validation)
predict_validation_prob = rf.predict_proba(X_validation)[:,1]

print('train accuracy :', accuracy_score(Y_train,predict_train_label))
print('validation accuracy :', accuracy_score(Y_validation,predict_validation_label))
print('\n')

print('train f1_score :', f1_score(Y_train,predict_train_label))
print('validation f1_score :', f1_score(Y_validation,predict_validation_label))
print('\n')

print('train recall_score :', recall_score(Y_train,predict_train_label))
print('validation recall_score :', recall_score(Y_validation,predict_validation_label))
print('\n')

print('train precision_score :', precision_score(Y_train,predict_train_label))
print('validation precision_score :', precision_score(Y_validation,predict_validation_label))
print('\n')

print('train auc :', roc_auc_score(Y_train,predict_train_proba))
print('validation auc :', roc_auc_score(Y_validation,predict_validation_prob))

train accuracy : 0.9965236154399425
validation accuracy : 0.7535279805352798


train f1_score : 0.9929731039496001
validation f1_score : 0.42736009044657997


train recall_score : 0.9927325581395349
validation recall_score : 0.3631123919308357


train precision_score : 0.9932137663596704
validation precision_score : 0.5192307692307693


train auc : 0.9998907607098495
validation auc : 0.740677513569584


In [97]:
# test데이터 마찬가지 위와 같은 방식
predict_test_label = rf.predict(x_test_dummies)
predict_test_proba = rf.predict_proba(x_test_dummies)[:,1]

## 정시 배송 여부 판단 (2회 기출)

In [116]:
import pandas as pd
import numpy as np

x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")

display(x_train.head())
display(y_train.head())

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1


In [117]:
x_train.info()
# 결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6598 non-null   int64 
 1   Warehouse_block      6598 non-null   object
 2   Mode_of_Shipment     6598 non-null   object
 3   Customer_care_calls  6598 non-null   object
 4   Customer_rating      6598 non-null   int64 
 5   Cost_of_the_Product  6598 non-null   int64 
 6   Prior_purchases      6598 non-null   int64 
 7   Product_importance   6598 non-null   object
 8   Gender               6598 non-null   object
 9   Discount_offered     6598 non-null   int64 
 10  Weight_in_gms        6598 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 567.1+ KB


In [118]:
x_train.nunique()
## ID만 삭제

ID                     6598
Warehouse_block           5
Mode_of_Shipment          3
Customer_care_calls       6
Customer_rating           5
Cost_of_the_Product     215
Prior_purchases           8
Product_importance        3
Gender                    2
Discount_offered         65
Weight_in_gms          3365
dtype: int64

In [119]:
x_train_drop = x_train.drop(columns='ID')
x_test_drop = x_test.drop(columns='ID')

In [120]:
## 더미화
x_train_dummies = pd.get_dummies(x_train_drop)
x_test_dummies = pd.get_dummies(x_test_drop)

# 컬럼 순서 동일하게
x_test_dummies = x_test_dummies[x_train_dummies.columns]

In [122]:
y = y_train['Reached.on.Time_Y.N']
y.head()

0    0
1    1
2    1
3    1
4    1
Name: Reached.on.Time_Y.N, dtype: int64

In [123]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(x_train_dummies, y, test_size=0.3, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(4618, 24) (1980, 24) (4618,) (1980,)


In [124]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [128]:
## 예측 & 성능평가
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

train_pred = rf.predict(X_train)
val_pred = rf.predict(X_val)
train_pred_proba = rf.predict_proba(X_train)[:,1]
val_pred_proba = rf.predict_proba(X_val)[:,1]

print('train_accuracy:', accuracy_score(y_train, train_pred))
print('val_accuracy:', accuracy_score(y_val, val_pred))
print('\n')

print('train_f1:', f1_score(y_train, train_pred))
print('val_f1:', f1_score(y_val, val_pred))
print('\n')

print('train_recall:', recall_score(y_train, train_pred))
print('val_recall:', recall_score(y_val, val_pred))
print('\n')

print('train_precision:', precision_score(y_train, train_pred))
print('val_precision:', precision_score(y_val, val_pred))
print('\n')

print('train roc_auc:', roc_auc_score(y_train, train_pred_proba))
print('val roc_auc:', roc_auc_score(y_val, val_pred_proba))

train_accuracy: 1.0
val_accuracy: 0.6757575757575758


train_f1: 1.0
val_f1: 0.7087114337568058


train_recall: 1.0
val_recall: 0.6568544995794785


train_precision: 1.0
val_precision: 0.7694581280788177


train roc_auc: 0.9999999999999999
val roc_auc: 0.7480863881832941


In [129]:
## test 예측
test_pred = rf.predict(x_test_dummies)
test_pred_proba = rf.predict_proba(x_test_dummies)[:,1]

pd.DataFrame({'ID':x_test.ID, 'Reached.on.Time_Y.N':test_pred}).to_csv("result.csv", index=False)
pd.DataFrame({'ID':x_test.ID, 'Reached.on.Time_Y.N':test_pred_proba}).to_csv("result2.csv", index=False)

## 성인 건강검진 데이터

In [10]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),...,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,구강검진수검여부,치아우식증유무,치석
0,0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,...,126.0,12.9,1.0,0.7,18.0,19.0,27.0,Y,0.0,Y
1,1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,...,127.0,12.7,1.0,0.6,22.0,19.0,18.0,Y,0.0,Y
2,2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,...,151.0,15.8,1.0,1.0,21.0,16.0,22.0,Y,0.0,N
3,3,M,40,165,70,88.0,1.5,1.5,1.0,1.0,...,226.0,14.7,1.0,1.0,19.0,26.0,18.0,Y,0.0,Y
4,4,F,40,155,60,86.0,1.0,1.0,1.0,1.0,...,107.0,12.5,1.0,0.6,16.0,14.0,22.0,Y,0.0,N


,ID,흡연상태
0,0,0
1,1,0
2,2,1
3,3,0
4,4,0


In [11]:
x_train.info()
# 결측치 없음
# object - 성별코드, 구강검진수검여부, 치석

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44553 entries, 0 to 44552
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           44553 non-null  int64  
 1   성별코드         44553 non-null  object 
 2   연령대코드(5세단위)  44553 non-null  int64  
 3   신장(5Cm단위)    44553 non-null  int64  
 4   체중(5Kg단위)    44553 non-null  int64  
 5   허리둘레         44553 non-null  float64
 6   시력(좌)        44553 non-null  float64
 7   시력(우)        44553 non-null  float64
 8   청력(좌)        44553 non-null  float64
 9   청력(우)        44553 non-null  float64
 10  수축기혈압        44553 non-null  float64
 11  이완기혈압        44553 non-null  float64
 12  식전혈당(공복혈당)   44553 non-null  float64
 13  총콜레스테롤       44553 non-null  float64
 14  트리글리세라이드     44553 non-null  float64
 15  HDL콜레스테롤     44553 non-null  float64
 16  LDL콜레스테롤     44553 non-null  float64
 17  혈색소          44553 non-null  float64
 18  요단백          44553 non-null  float64
 19  혈청크레

In [12]:
x_train_drop = x_train.drop(columns='ID')
x_test_drop = x_test.drop(columns='ID')

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

x_train_drop['성별코드'] = le.fit_transform(x_train_drop['성별코드'])
x_train_drop['구강검진수검여부'] = le.fit_transform(x_train_drop['구강검진수검여부'])
x_train_drop['치석'] = le.fit_transform(x_train_drop['치석'])

x_test_drop['성별코드'] = le.fit_transform(x_test_drop['성별코드'])
x_test_drop['구강검진수검여부'] = le.fit_transform(x_test_drop['구강검진수검여부'])
x_test_drop['치석'] = le.fit_transform(x_test_drop['치석'])
x_train_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44553 entries, 0 to 44552
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   성별코드         44553 non-null  int32  
 1   연령대코드(5세단위)  44553 non-null  int64  
 2   신장(5Cm단위)    44553 non-null  int64  
 3   체중(5Kg단위)    44553 non-null  int64  
 4   허리둘레         44553 non-null  float64
 5   시력(좌)        44553 non-null  float64
 6   시력(우)        44553 non-null  float64
 7   청력(좌)        44553 non-null  float64
 8   청력(우)        44553 non-null  float64
 9   수축기혈압        44553 non-null  float64
 10  이완기혈압        44553 non-null  float64
 11  식전혈당(공복혈당)   44553 non-null  float64
 12  총콜레스테롤       44553 non-null  float64
 13  트리글리세라이드     44553 non-null  float64
 14  HDL콜레스테롤     44553 non-null  float64
 15  LDL콜레스테롤     44553 non-null  float64
 16  혈색소          44553 non-null  float64
 17  요단백          44553 non-null  float64
 18  혈청크레아티닌      44553 non-null  float64
 19  (혈청지

In [15]:
## modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
rf = RandomForestClassifier(random_state=0)

y = y_train['흡연상태']
X_train, X_val, y_train, y_val = train_test_split(x_train_drop, y, test_size=0.3, random_state=0, stratify=y)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(31187, 25) (13366, 25) (31187,) (13366,)


In [16]:
rf.fit(X_train, y_train)  # 학습

RandomForestClassifier(random_state=0)

In [17]:
## 예측, 성능평가
train_pred = rf.predict(X_train)
val_pred = rf.predict(X_val)

train_pred_proba = rf.predict_proba(X_train)[:,1]
val_pred_proba = rf.predict_proba(X_val)[:,1]

from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score
print('==== train set ====')
print('accuracy :', accuracy_score(y_train, train_pred))
print('recall :', recall_score(y_train, train_pred))
print('precision :', precision_score(y_train, train_pred))
print('roc_auc :', roc_auc_score(y_train, train_pred_proba))
print('\n')
print('==== test set ====')
print('accuracy :', accuracy_score(y_val, val_pred))
print('recall :', recall_score(y_val, val_pred))
print('precision :', precision_score(y_val, val_pred))
print('roc_auc :', roc_auc_score(y_val, val_pred_proba))

==== train set ====
accuracy : 0.9999679353576811
recall : 0.9999126332343177
precision : 1.0
roc_auc : 1.0


==== test set ====
accuracy : 0.7507107586413287
recall : 0.6960864247859764
precision : 0.6497336377473364
roc_auc : 0.8357466830310548


In [20]:
## test set
pred = rf.predict(x_test_drop)
pred_proba = rf.predict_proba(x_test_drop)[:,1]

pd.DataFrame({'ID':x_test.ID, 'pred':pred}).to_csv('result.csv', index=False)
pd.DataFrame({'ID':x_test.ID, 'pred':pred_proba}).to_csv('result2.csv', index=False)

## 자동차 보험가입 예측데이터

In [21]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,id
0,0,Female,23,1,8.0,0,< 1 Year,Yes,61354.0,152.0,235,NaN
1,1,Male,27,1,28.0,1,< 1 Year,No,38036.0,152.0,207,NaN
2,2,Female,23,1,45.0,0,< 1 Year,Yes,25984.0,152.0,217,NaN
3,3,Male,22,1,46.0,0,< 1 Year,No,39499.0,152.0,277,NaN
4,4,Male,32,1,30.0,1,< 1 Year,No,38771.0,152.0,251,NaN


,ID,Response
0,0,0
1,1,0
2,2,1
3,3,0
4,4,0


In [23]:
x_train.info()
# object : Gender, Vehicle_Age, Vehicle_Damage
# id drop
## 결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304887 entries, 0 to 304886
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    304887 non-null  int64  
 1   Gender                304887 non-null  object 
 2   Age                   304887 non-null  int64  
 3   Driving_License       304887 non-null  int64  
 4   Region_Code           304887 non-null  float64
 5   Previously_Insured    304887 non-null  int64  
 6   Vehicle_Age           304887 non-null  object 
 7   Vehicle_Damage        304887 non-null  object 
 8   Annual_Premium        304887 non-null  float64
 9   Policy_Sales_Channel  304887 non-null  float64
 10  Vintage               304887 non-null  int64  
 11  id                    0 non-null       float64
dtypes: float64(4), int64(5), object(3)
memory usage: 27.9+ MB


In [22]:
x_train.nunique()

ID                      304887
Gender                       2
Age                         66
Driving_License              2
Region_Code                 53
Previously_Insured           2
Vehicle_Age                  3
Vehicle_Damage               2
Annual_Premium           46430
Policy_Sales_Channel       155
Vintage                    290
id                           0
dtype: int64

In [26]:
x_train_drop = x_train.drop(columns='id')
x_test_drop = x_test.drop(columns='id')

x_train_dummies = pd.get_dummies(x_train_drop)
x_test_dummies = pd.get_dummies(x_test_drop)

x_test_dummies = x_test_dummies[x_train_dummies.columns]
x_train_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304887 entries, 0 to 304886
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     304887 non-null  int64  
 1   Age                    304887 non-null  int64  
 2   Driving_License        304887 non-null  int64  
 3   Region_Code            304887 non-null  float64
 4   Previously_Insured     304887 non-null  int64  
 5   Annual_Premium         304887 non-null  float64
 6   Policy_Sales_Channel   304887 non-null  float64
 7   Vintage                304887 non-null  int64  
 8   Gender_Female          304887 non-null  bool   
 9   Gender_Male            304887 non-null  bool   
 10  Vehicle_Age_1-2 Year   304887 non-null  bool   
 11  Vehicle_Age_< 1 Year   304887 non-null  bool   
 12  Vehicle_Age_> 2 Years  304887 non-null  bool   
 13  Vehicle_Damage_No      304887 non-null  bool   
 14  Vehicle_Damage_Yes     304887 non-nu

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

rf = RandomForestClassifier(random_state=0)

y = y_train['Response']
X_train, X_val, y_train, y_val = train_test_split(x_train_dummies, y, test_size=0.3, random_state=0, stratify=y)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

rf.fit(X_train, y_train)

(213420, 15) (91467, 15) (213420,) (91467,)


RandomForestClassifier(random_state=0)

In [28]:
train_pred = rf.predict(X_train)
test_pred = rf.predict(X_val)
train_pred_proba = rf.predict_proba(X_train)[:,1]
test_pred_proba = rf.predict_proba(X_val)[:,1]

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
print('== train ==')
print('accuracy_score :', accuracy_score(y_train, train_pred))
print('f1_score :', f1_score(y_train, train_pred))
print('recall_score :', recall_score(y_train, train_pred))
print('precision_score :', precision_score(y_train, train_pred))
print('roc_auc_score :', roc_auc_score(y_train, train_pred_proba))
print('\n')
print('== test ==')
print('accuracy_score :', accuracy_score(y_val, test_pred))
print('f1_score :', f1_score(y_val, test_pred))
print('recall_score :', recall_score(y_val, test_pred))
print('precision_score :', precision_score(y_val, test_pred))
print('roc_auc_score :', roc_auc_score(y_val, test_pred_proba))

== train ==
accuracy_score : 0.9999718864211414
f1_score : 0.9998852947923836
recall_score : 0.9997706158963184
precision_score : 1.0
roc_auc_score : 1.0


== test ==
accuracy_score : 0.8712322476958904
f1_score : 0.119994022713688
recall_score : 0.07162608152707163
precision_score : 0.36953520478601015
roc_auc_score : 0.8387887030612735


In [29]:
pred = rf.predict(x_test_dummies)
pred_proba = rf.predict_proba(x_test_dummies)[:,1]

pd.DataFrame({'ID':x_test.ID, 'Response':pred}).to_csv('result.csv', index=False)
pd.DataFrame({'ID':x_test.ID, 'Response-proba':pred_proba}).to_csv('result2.csv', index=False)

## 비행탑승 경험 만족도 데이터

In [39]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,id
0,0,Female,Loyal Customer,54,Personal Travel,Eco,1068,3,4,3,...,5,5,3,5,3,5,3,47,22.0,NaN
1,2,Male,Loyal Customer,20,Personal Travel,Eco,1546,4,4,4,...,4,3,3,4,4,4,4,5,2.0,NaN
2,3,Male,Loyal Customer,59,Business travel,Business,2962,0,4,0,...,1,1,1,1,5,1,4,54,46.0,NaN
3,4,Male,Loyal Customer,35,Business travel,Eco Plus,106,5,4,4,...,5,2,1,5,4,4,5,130,121.0,NaN
4,5,Female,Loyal Customer,9,Business travel,Business,2917,3,3,3,...,4,4,4,5,4,3,4,0,0.0,NaN


,ID,satisfaction
0,0,neutral or dissatisfied
1,2,neutral or dissatisfied
2,3,satisfied
3,4,satisfied
4,5,satisfied


In [31]:
x_train.info()
# Arrival Delay in Minutes 결측치 있음
# object : Gender, Customer Type, Type of Travel, Class

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83123 entries, 0 to 83122
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID                                 83123 non-null  int64  
 1   Gender                             83123 non-null  object 
 2   Customer Type                      83123 non-null  object 
 3   Age                                83123 non-null  int64  
 4   Type of Travel                     83123 non-null  object 
 5   Class                              83123 non-null  object 
 6   Flight Distance                    83123 non-null  int64  
 7   Inflight wifi service              83123 non-null  int64  
 8   Departure/Arrival time convenient  83123 non-null  int64  
 9   Ease of Online booking             83123 non-null  int64  
 10  Gate location                      83123 non-null  int64  
 11  Food and drink                     83123 non-null  int

In [32]:
x_train[['Gender', 'Customer Type', 'Type of Travel', 'Class']].nunique()

Gender            2
Customer Type     2
Type of Travel    2
Class             3
dtype: int64

In [40]:
x_train_drop = x_train.drop(columns=['ID','id'])
x_test_drop = x_test.drop(columns=['ID','id'])
x_train_drop['Arrival Delay in Minutes'].fillna(x_train_drop['Arrival Delay in Minutes'].mean(), inplace=True)
x_test_drop['Arrival Delay in Minutes'].fillna(x_test_drop['Arrival Delay in Minutes'].mean(), inplace=True)

x_train_dummies = pd.get_dummies(x_train_drop)
x_test_dummies = pd.get_dummies(x_test_drop)
x_test_dummies = x_test_dummies[x_train_dummies.columns]

#######
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train['satisfaction'] = le.fit_transform(y_train['satisfaction'])
print('== y 라벨 인코딩 결과 == \n', le.fit_transform(['neutral or dissatisfied', 'satisfied']))

#######
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
rf = RandomForestClassifier(random_state=0)
y = y_train['satisfaction']

X_train, X_val, y_train, y_val = train_test_split(x_train_dummies, y, test_size=0.3, stratify=y, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

rf.fit(X_train, y_train)

#######
train_pred = rf.predict(X_train)
train_pred_proba = rf.predict_proba(X_train)[:,1]
val_pred = rf.predict(X_val)
val_pred_proba = rf.predict_proba(X_val)[:,1]

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
print('== train == ')
print('accuracy_score', accuracy_score(y_train, train_pred))
print('f1_score', f1_score(y_train, train_pred))
print('recall_score', recall_score(y_train, train_pred))
print('precision_score', precision_score(y_train, train_pred))
print('roc_auc_score', roc_auc_score(y_train, train_pred_proba))
print('== val ==')
print('accuracy_score', accuracy_score(y_val, val_pred))
print('f1_score', f1_score(y_val, val_pred))
print('recall_score', recall_score(y_val, val_pred))
print('precision_score', precision_score(y_val, val_pred))
print('roc_auc_score', roc_auc_score(y_val, val_pred_proba))


########
pred = rf.predict(x_test_dummies)
pred_proba = rf.predict_proba(x_test_dummies)[:,1]
pd.DataFrame({'ID':x_test.ID, 'satisfaction':pred}).to_csv('result.csv', index=False)
pd.DataFrame({'ID':x_test.ID, 'satisfaction':pred_proba}).to_csv('result2.csv', index=False)

== y 라벨 인코딩 결과 == 
 [0 1]
(58186, 27) (24937, 27) (58186,) (24937,)
== train == 
accuracy_score 1.0
f1_score 1.0
recall_score 1.0
precision_score 1.0
roc_auc_score 1.0
== val ==
accuracy_score 0.9611019769819946
f1_score 0.9543958627174424
recall_score 0.9392929853784935
precision_score 0.9699923547400612
roc_auc_score 0.9936016722402903


## 수질 음용성 여부 데이터

In [47]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,0,8.662710,173.531947,20333.079495,5.636388,439.787938,459.633120,16.283311,89.924253,5.120103
1,1,NaN,226.270824,15380.124079,6.661474,NaN,392.558205,14.083110,50.286395,4.516870
2,2,7.583770,217.283262,36343.407055,8.532726,375.964391,393.877683,17.442301,77.722257,3.642289
3,3,6.584813,182.375456,24723.106296,6.238920,NaN,414.350751,17.582615,78.213738,4.404132
4,4,7.179864,180.854211,10859.553752,8.263503,341.302486,358.056264,12.065317,83.329918,3.878447


,ID,Potability
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0


In [44]:
print(x_train.info(), x_test.info())
## 결측치 : ph, Sulfate, Trihalomethanes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2620 entries, 0 to 2619
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2620 non-null   int64  
 1   ph               2225 non-null   float64
 2   Hardness         2620 non-null   float64
 3   Solids           2620 non-null   float64
 4   Chloramines      2620 non-null   float64
 5   Sulfate          2003 non-null   float64
 6   Conductivity     2620 non-null   float64
 7   Organic_carbon   2620 non-null   float64
 8   Trihalomethanes  2488 non-null   float64
 9   Turbidity        2620 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 204.8 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               656 non-null    int64  
 1   ph               549 non-null    float64


In [48]:
# drop
x_train_drop = x_train.drop(columns='ID')
x_test_drop = x_test.drop(columns='ID')

# 결측치 평균값으로 채우기
x_train_drop['ph'].fillna(x_train_drop['ph'].mean(), inplace=True)
x_train_drop['Sulfate'].fillna(x_train_drop['Sulfate'].mean(), inplace=True)
x_train_drop['Trihalomethanes'].fillna(x_train_drop['Trihalomethanes'].mean(), inplace=True)

x_test_drop['ph'].fillna(x_test_drop['ph'].mean(), inplace=True)
x_test_drop['Sulfate'].fillna(x_test_drop['Sulfate'].mean(), inplace=True)
x_test_drop['Trihalomethanes'].fillna(x_test_drop['Trihalomethanes'].mean(), inplace=True)

# split
from sklearn.model_selection import train_test_split
y = y_train['Potability']
X_train, X_val, y_train, y_val = train_test_split(x_train_drop, y, test_size=0.3, stratify=y, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# modeling
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

# predict
train_pred = rf.predict(X_train)
train_pred_proba = rf.predict_proba(X_train)[:,1]
val_pred = rf.predict(X_val)
val_pred_proba = rf.predict_proba(X_val)[:,1]

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
print('\n')
print('== train ==')
print('accuracy_score :', accuracy_score(y_train, train_pred))
print('f1_score :', f1_score(y_train, train_pred))
print('recall_score :', recall_score(y_train, train_pred))
print('precision_score :', precision_score(y_train, train_pred))
print('roc_auc_score :', roc_auc_score(y_train, train_pred_proba))
print('\n')
print('== val ==')
print('accuracy_score :', accuracy_score(y_val, val_pred))
print('f1_score :', f1_score(y_val, val_pred))
print('recall_score :', recall_score(y_val, val_pred))
print('precision_score :', precision_score(y_val, val_pred))
print('roc_auc_score :', roc_auc_score(y_val, val_pred_proba))

pred = rf.predict(x_test_drop)
pred_proba = rf.predict_proba(x_test_drop)[:,1]
pd.DataFrame({'ID':x_test.ID, 'Potability':pred}).to_csv('result.csv', index=False)
pd.DataFrame({'ID':x_test.ID, 'Potability-proba':pred_proba}).to_csv('result2.csv', index=False)

(1834, 9) (786, 9) (1834,) (786,)


== train ==
accuracy_score : 1.0
f1_score : 1.0
recall_score : 1.0
precision_score : 1.0
roc_auc_score : 1.0


== val ==
accuracy_score : 0.638676844783715
f1_score : 0.3879310344827587
recall_score : 0.2931596091205212
precision_score : 0.5732484076433121
roc_auc_score : 0.620772782602191


## 약물 분류 데이터

In [55]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,Age,Sex,BP,Cholesterol,Na_to_K
0,0,36,F,NORMAL,HIGH,16.753
1,1,47,F,LOW,HIGH,11.767
2,2,69,F,NORMAL,HIGH,10.065
3,3,35,M,LOW,NORMAL,9.170
4,4,49,M,LOW,NORMAL,11.014


,ID,Drug
0,0,0
1,1,3
2,2,4
3,3,4
4,4,4


In [50]:
print(x_train.info(), x_test.info())
# 결측치 없음
# object : Sex, BP, Cholesterol > 더미화

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           157 non-null    int64  
 1   Age          157 non-null    int64  
 2   Sex          157 non-null    object 
 3   BP           157 non-null    object 
 4   Cholesterol  157 non-null    object 
 5   Na_to_K      157 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 7.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           43 non-null     int64  
 1   Age          43 non-null     int64  
 2   Sex          43 non-null     object 
 3   BP           43 non-null     object 
 4   Cholesterol  43 non-null     object 
 5   Na_to_K      43 non-null     float64
dtypes: float64(1), int64(2), object(3)
memory usage: 2.1+ K

In [57]:
print(y_train.info())

<class 'pandas.core.series.Series'>
Index: 109 entries, 145 to 57
Series name: Drug
Non-Null Count  Dtype
--------------  -----
109 non-null    int64
dtypes: int64(1)
memory usage: 1.7 KB
None


In [51]:
print(x_train.nunique())

ID             157
Age             55
Sex              2
BP               3
Cholesterol      2
Na_to_K        156
dtype: int64


In [ ]:
# drop ID
x_train_drop = x_train.drop(columns='ID')
x_test_drop = x_test.drop(columns='ID')

# 더미화
x_train_dummies = pd.get_dummies(x_train_drop)
x_test_dummies = pd.get_dummies(x_test_drop)
x_test_dummies = x_test_dummies[x_train_dummies.columns]

# split
y = y_train['Drug']
X_train, X_val, y_train, y_val = train_test_split(x_train_dummies, y, test_size=0.3, random_state=0, stratify=y)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

In [59]:
# pred
train_pred = rf.predict(X_train)
train_pred_proba = rf.predict_proba(X_train)[:,1]
val_pred = rf.predict(X_val)
val_pred_proba = rf.predict_proba(X_val)[:,1]

print('\n')
print('== train ==')
print('accuracy_score :', accuracy_score(y_train, train_pred))
# print('f1_score :', f1_score(y_train, train_pred))
# print('recall_score :', recall_score(y_train, train_pred))
# print('precision_score :', precision_score(y_train, train_pred))
# print('roc_auc_score :', roc_auc_score(y_train, train_pred_proba))
print('\n')
print('== val ==')
print('accuracy_score :', accuracy_score(y_val, val_pred))
# print('f1_score :', f1_score(y_val, val_pred))
# print('recall_score :', recall_score(y_val, val_pred))
# print('precision_score :', precision_score(y_val, val_pred))
# print('roc_auc_score :', roc_auc_score(y_val, val_pred_proba))

pred = rf.predict(x_test_dummies)
pred_proba = rf.predict_proba(x_test_dummies)[:,1]
pd.DataFrame({'ID':x_test.ID, 'Drug':pred}).to_csv('result.csv', index=False)
pd.DataFrame({'ID':x_test.ID, 'Drug-proba':pred_proba}).to_csv('result2.csv', index=False)



== train ==
accuracy_score : 1.0


== val ==
accuracy_score : 1.0
